### 自定义层

In [2]:
from turtle import forward
from matplotlib.pyplot import cla
import torch
import torch.nn.functional as F
from torch import nn

class CenterLayer(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, X):
        return X-X.mean()
        

测试一下这个自定义层能否工作

In [3]:
layer=CenterLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到更复杂的模型中

In [5]:
net=nn.Sequential(nn.Linear(8, 128), CenterLayer())

向该网络发送随机数据后，检查均值是否为0

In [7]:
Y=net(torch.rand(4, 8))
Y.mean()

tensor(-1.0710e-08, grad_fn=<MeanBackward0>)

定义具有参数的层， 这些参数可以通过训练进行调整。 我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。 比如管理访问、初始化、共享、保存和加载模型参数。 这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。
实现自定义版本的全连接层。 回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，我们使用修正线性单元作为激活函数。 该层需要输入参数：in_units和units，分别表示输入数和输出数。

In [8]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units) -> None:
        super().__init__()
        self.weight=nn.Parameter(torch.rand(in_units, units))
        self.bias=nn.Parameter(torch.rand(units,))
    def forward(self, X):
        linear=torch.matmul(X, self.weight.data)+self.bias.data
        return F.relu(linear)

In [9]:
linear= MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[0.1028, 0.0748, 0.7882],
        [0.9855, 0.1542, 0.9713],
        [0.6062, 0.1882, 0.8951],
        [0.8469, 0.0924, 0.5475],
        [0.2782, 0.4603, 0.7395]], requires_grad=True)

使用自定义层直接执行向前传播计算

In [18]:
linear(torch.rand(5, 5))

tensor([[1.4297, 0.8390, 2.7308],
        [1.0829, 0.5634, 2.0891],
        [1.0162, 0.3973, 1.4196],
        [2.2877, 0.7737, 3.2635],
        [1.5343, 0.5623, 2.0318]])